# Following the procedure from UN-SPIDER radar-based-flood mapping 

1. Download sentinel-1 image(s) from Copernicus Open Acces Hub 
2. post-process images (noise filtering, etc.) using ESA's 'SNAP' tool package 
3. Create a binary map: "water" vs "non-water" 
4. Use GlobCover map to determine location of pernimant water bodies and use difference with step 3 to end up with "flooded area" vs "not flooded"


## packages to install (on top of docker image with snappy)

```
pip install sentinelsat. 
conda install geopandas
pip install rasterio
conda install -c anaconda ipyleaflet
conda install -c anaconda ipywidgets
conda install scikit-image
```


**sharing the entire environment (takes care of conda vs pip packages)** 

1. When done building all the codesave environment specs to a file:
    ```
    conda env export > environment.yml
    ```
**note: might want to exclude jupyter-lab? unless needed for the widgets and we are not using any other solution** 

2. when building docker image, simply build the conda environment including all required packages: 
    ```
    conda env create -f environment.yml 
    ```


In [1]:
import snappy                                                           # postprocessing tools from ESA for SAR satellite images 
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt       # access to Copernicus Open Access Hub 
import rasterio                                                         # dealing with raster images + masking with shapefiles 
import geopandas  as gpd                                                # GeoDataFrames


from datetime import date                     # dates, times and intervalls
import ipyleaflet                             # visualization
import ipywidgets
import json 
import zipfile 
import skimage.filters                        # threshold calculation


import numpy as np 
import time 

# Connect to Copernicus Open Access Hub 

In [2]:
username = 'jacopo_margutti'
password = 'Room4Copernicus'
api = SentinelAPI(username , password)

# Region of interest 

#### possible extentions 

* woosterheert used GDACS database to retreive event locations. Do we want to have this? 
* Get shapefile online and let user simply supply the country (code) 

In [3]:
def bounding_box(polygon):
    '''
    use built-in method of geopandas to get bounding box when supplying Geo DataFrame 
    
    Syntax does not make clear I am calling Geopandas --> the function works on a shapely geometry object. 
    '''    
    xmin, ymin, xmax, ymax = polygon.bounds 
    return xmin, ymin, xmax, ymax
    

def create_polygon_from_bbox(bbox_in):
    """
    Turns a list of bounding box coordinates into a list of polygon points by creating a closed loop that defines the contours of the bounding box
    This can be used for query. 
    """
    xmin, ymin, xmax, ymax = bbox_in 
    polygon =[(ymin, xmin),
              (ymax, xmin),
              (ymax, xmax),
              (ymin, xmax),
              (ymin, xmin)
             ]
    return polygon






shapeData = gpd.read_file('admin_boundaries/zwe_admbnda_adm0_zimstat_ocha_20180911.shp')

for geometry in shapeData['geometry']:
    bbox = bounding_box(geometry)
    pol = create_polygon_from_bbox(bbox)

#### check bounding box on map 

In [4]:
download_map = ipyleaflet.Map(zoom = 1.4)
download_map.add_control(ipyleaflet.SearchControl(
    position = 'topleft',
    url = 'https://nominatim.openstreetmap.org/search?format=json&q={s}',
    zoom = 5))
download_map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))







display(download_map)

# show AOI on map according to JSON data
with open("zimbabwe.geojson",'r') as geoJSON:
    data_json = json.load(geoJSON)

aoi = ipyleaflet.GeoJSON(data = data_json, style = {'color' : 'green'})
download_map.center = (aoi.data['features'][0]['geometry']['coordinates'][0][0][1],
                       aoi.data['features'][0]['geometry']['coordinates'][0][0][0])
download_map.zoom = 4.5 
download_map.add_layer(aoi)


aoi_bbox = ipyleaflet.Polygon(locations = pol, color="red", fill_color="white")
download_map.add_layer(aoi_bbox)






Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

#### Retreiving shapefile from GeoNode

Main issues:
* non-uniform naming of files 
* not always shapefile available 
* URL is outdated I think 

In [ ]:
from owslib.wfs import WebFeatureService


def download_shapefile(mySettings):
    '''
    Retrieve shapefile data from Geonode server.
    Store a 'GeoJSON' file (GeoDataFrame) in temporary folder that contains the
    IDs of the boundaries and the shapes/polygons.

    :param mySettings:
    :return:
    '''

    # --- Ask if possible to make one uniform naming for shapefiles ---
    country = mySettings.country
    admin_lvl = mySettings.admin_lvl
    layer_id = 'geonode:'+ '_'.join([country, admin_lvl, 'mapshaper','reproj'])

    wfs_url = 'https://geonode.510.global/geoserver/geonode_old/ows'
    wfs = WebFeatureService(wfs_url)
    wfs = WebFeatureService(wfs_url, version='1.0.0')

    # Get the actual data
    binary_data = wfs.getfeature(typename=layer_id, outputFormat='application/json')

    # write to file
    shapefile_filename =layer_id.replace('geonode:', '') + '.geojson'
    shapefile_filename = os.path.join(mySettings.temporary_folder, shapefile_filename )
#     with open(shapefile_filename, 'wb') as shapefile:
    with open(shapefile_filename, 'w') as shapefile:
        shapefile.write(binary_data.read())
    return shapefile_filename


# Query available images at Copernicus Open Access Hub 


Add option to enter your own dates of interest?
I like woosterheerts' default to query for the last 30 days 

In [5]:
def get_available_satellite_images(api_in, coordinates_in, coordinates_type='POLY'):
    """
    Queries the sentinel api for available images in the past 30 days for a specified region for Sentinel-1.
    Region can be specified either by the coordinates of a point (coordinates_type='POINT') or a list of polygon points.
    The area in the query needs to be provided as in:
    https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry
    After querying it filters for the GRD images and sorts them by ingestion date
    Function returns a dataframe with all the available image id's and their properties
    """
    
    if coordinates_type == 'POLY':
        poly = coordinates_in
        area = "POLYGON(({}))".format(",".join(["{} {}".format(p[0], p[1]) for p in poly]))
    else:
        point = coordinates_in
        area = "POINT({} {})".format(point[0], point[1])

    products = api_in.query(date=('NOW-30DAYS', 'NOW'),
                            platformname='Sentinel-1',
                            producttype = 'GRD',
                            area=area)

    products_df = api_in.to_dataframe(products)
    products_df.sort_values('ingestiondate', ascending=False, inplace=True)
    return products_df

In [6]:
products = get_available_satellite_images(api, pol)

Querying products:  49%|####8     | 100/205 [00:00<?, ? products/s]

In [7]:
products.head()

,title,link,link_alternative,link_icon,summary,ondemand,beginposition,endposition,ingestiondate,missiondatatakeid,...,platformname,platformidentifier,instrumentname,instrumentshortname,productclass,polarisationmode,acquisitiontype,gmlfootprint,footprint,uuid
e87893d5-8299-4739-b339-d915f9469a39,S1B_IW_GRDH_1SDV_20210806T070209_20210806T0702...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-08-06T07:02:09.946Z, Instrument: SA...",false,2021-08-06 07:02:09.946,2021-08-06 07:02:34.944,2021-08-06 08:18:11.414,219860,...,Sentinel-1,2016-025A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-15.065257 27.05961, -14.73430...",e87893d5-8299-4739-b339-d915f9469a39
cb5beee4-7760-4caf-8920-2de709a46a0a,S1B_IW_GRDH_1SDV_20210806T070234_20210806T0703...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-08-06T07:02:34.945Z, Instrument: SA...",false,2021-08-06 07:02:34.945,2021-08-06 07:03:02.247,2021-08-06 08:12:15.188,219860,...,Sentinel-1,2016-025A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-15.422532 25.412209, -15.0652...",cb5beee4-7760-4caf-8920-2de709a46a0a
2c3aa57e-eeae-483e-bc0d-a07152058773,S1B_IW_GRDH_1SDV_20210806T070119_20210806T0701...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-08-06T07:01:19.945Z, Instrument: SA...",false,2021-08-06 07:01:19.945,2021-08-06 07:01:44.943,2021-08-06 08:06:18.627,219860,...,Sentinel-1,2016-025A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-14.399137 30.074718, -14.0593...",2c3aa57e-eeae-483e-bc0d-a07152058773
c1c58f98-0092-422f-9f80-f68e8a31f437,S1B_IW_GRDH_1SDV_20210806T070054_20210806T0701...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-08-06T07:00:54.946Z, Instrument: SA...",false,2021-08-06 07:00:54.946,2021-08-06 07:01:19.944,2021-08-06 08:00:10.906,219860,...,Sentinel-1,2016-025A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-14.059367 31.58127, -13.71443...",c1c58f98-0092-422f-9f80-f68e8a31f437
1a392cc8-ae2c-4008-8bed-2988de978a27,S1B_IW_GRDH_1SDV_20210806T070144_20210806T0702...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-08-06T07:01:44.945Z, Instrument: SA...",false,2021-08-06 07:01:44.945,2021-08-06 07:02:09.944,2021-08-06 07:54:13.519,219860,...,Sentinel-1,2016-025A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-14.734282 28.567427, -14.3991...",1a392cc8-ae2c-4008-8bed-2988de978a27


# Download images 

### make folder (can be temporary) 

In [ ]:
import os

# temp folder to store all downloads into 
downloadFldr = os.path.join(os.getcwd(), 'sentinel_downloads')
if not os.path.exists(downloadFldr):
    os.mkdir(downloadFldr)

# at the end of code: 
os.rmdir(downloadFldr)

In [ ]:
tile_id = products['uuid'][0]
product_info = api.download(id = tile_id, directory_path=downloadFldr)

In [ ]:
product_info

# Preprocess obtained Sentinel-1 product (raw satelite data downloaded)

Use snappy package to process images according to UN-Spider protocol 

In [8]:
import os 
downloadFldr = os.path.join(os.getcwd(), 'sentinel_downloads')

for root, _ , files in os.walk(downloadFldr):
    for file in files:
        file_sentinel = os.path.join(root, file)

print(file_sentinel)

/home/jovyan/flood_extent_mapping/sentinel_downloads/S1A_IW_GRDH_1SDV_20210805T071151_20210805T071216_039091_049CD6_EF61.zip


In [9]:
def get_sourceBands(polarisations):
    
    # use alphabetical order to be consistent with UN-spider code (might not be strictly necessary)
    listOfPolarisations = polarisations.split()
    listOfPolarisations.sort()
    return ','.join([f"Amplitude_{polarisation},Intensity_{polarisation}" for polarisation in listOfPolarisations])
        
    
sourceBands = get_sourceBands(products['polarisationmode'].iloc[0])
print(sourceBands)

Amplitude_VH,Intensity_VH,Amplitude_VV,Intensity_VV


#### processing steps 

In [10]:
import time 

def apply_orbit_file(S1):
    parameters = snappy.HashMap()
    # continue with calculation in case no orbit file is available yet
    parameters.put('continueOnFail', True)
    
    S1_Orb = snappy.GPF.createProduct('Apply-Orbit-File', parameters, S1)
    return S1_Orb 


def thermal_noise_removal(S1):
    parameters = snappy.HashMap()
    parameters.put('copyMetadata', True)
    parameters.put('removeThermalNoise', True)
    S1_Thm = snappy.GPF.createProduct('ThermalNoiseRemoval', parameters, S1)
    return S1_Thm

def radiometric_callibration(S1):
    parameters = snappy.HashMap()
    parameters.put('outputSigmaBand', True)
    S1_Cal = snappy.GPF.createProduct('Calibration', parameters, S1)
    return S1_Cal


def speckle_filtering(S1, filtersize = (5,5)):
    
    filtersize_x, filtersize_y = filtersize
    
    parameters = snappy.HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', filtersize_x)
    parameters.put('filterSizeY', filtersize_y)
    S1_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1)
    return S1_Spk


def terrain_correction(S1):
    S1_db = snappy.GPF.createProduct('LinearToFromdB', snappy.HashMap(), S1)
    parameters = snappy.HashMap()
    parameters.put('demName', 'SRTM 1Sec HGT')
    parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
    parameters.put('pixelSpacingInMeter', 10.0)
    parameters.put('nodataValueAtSea', False)
    parameters.put('saveSelectedSourceBand', True)
    S1_TC = snappy.GPF.createProduct('Terrain-Correction', parameters, S1_db)
    return S1_TC 


### To avoid memory errors: Do entire processing within a function to not have all these S1-products in memory (global variables)

In [11]:
def preprocess_sentinel_1(file_sentinel_product):
    
    # ---- load Sentinel product ---- 
    SentinelProduct = snappy.ProductIO.readProduct(file_sentinel)
    
    
    # --- Oribit File (if available) ----
    print('1. Apply Orbit File:          ', end='', flush=True)
    start_time = time.time()
    AppliedOrbitFile = apply_orbit_file(S1 = SentinelProduct)
    del 
    
    
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)
    
    # --- Thermal Noise Removal ---- 
    print('2. Thermal Noise Removal:     ', end='', flush=True)
    start_time = time.time()
    ThermalNoiseRemoved = thermal_noise_removal(S1 = AppliedOrbitFile)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)
    
    # ---- Radiometric Calibration --- 
    print('3. Radiometric Calibration:    ', end='', flush=True)
    start_time = time.time()
    Calibrated = radiometric_callibration(S1 = ThermalNoiseRemoved)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # ---- Speckle Filtering ---- 
    print('4. Speckle Filtering:    ', end='', flush=True)
    start_time = time.time()
    SpeckleFiltered = speckle_filtering(S1 = Calibrated)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)


    # ---- Terrain Correction --- 
    print('5. Terrain Correction:        ', end='', flush=True)
    start_time = time.time()
    TerrainCorrected = terrain_correction(S1 = SpeckleFiltered)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)
    return TerrainCorrected


# determine flood extend
UN-spider protocol based on thresholding intensity values 


**Again, use local variables to not overlaod memory** 

In [12]:

def create_globcover(S1):
    parameters = snappy.HashMap()
    parameters.put('landCoverNames', 'GlobCover')
    GlobCover = snappy.GPF.createProduct('AddLandCover', parameters, S1)
    return GlobCover



def getThreshold(S1_band):
    # read band
    w = S1_band.getRasterWidth()
    h = S1_band.getRasterHeight()
    band_data = np.zeros(w * h, np.float32)
    S1_band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h * w

    # calculate threshold using Otsu method
    threshold_otsu = skimage.filters.threshold_otsu(band_data)
    # calculate threshold using minimum method
    threshold_minimum = skimage.filters.threshold_minimum(band_data)
    # get number of pixels for both thresholds
    numPixOtsu = len(band_data[abs(band_data - threshold_otsu) < 0.1])
    numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

    # if number of pixels at minimum threshold is less than 1% of number of pixels at Otsu threshold
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        # adjust band data according
        if threshold_otsu < threshold_minimum:
            band_data = band_data[band_data < threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
        else:
            band_data = band_data[band_data > threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
    
        numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

    # select final threshold
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        threshold = threshold_otsu
    else:
        threshold = threshold_minimum

    return threshold

def binarization(S1_product, expressions):

    BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')
    targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(expressions))

    # loop through bands
    for i in range(len(expressions)):
        targetBand = BandDescriptor()
        targetBand.name = '%s' % S1_product.getBandNames()[i]
        targetBand.type = 'float32'
        targetBand.expression = expressions[i]
        targetBands[i] = targetBand
    
    parameters = snappy.HashMap()
    parameters.put('targetBands', targetBands)    
    mask = snappy.GPF.createProduct('BandMaths', parameters, S1_product)

    return mask

In [13]:
def binary_flood_mask(S1):    
    print('6. Binarization:              ', end='', flush=True)
    start_time = time.time()
    
    # Permanent water bodies from GlobCover map
    GlobCover = create_globcover(S1)

    
    # Threshold calculations to separate water pixels from non-water pixels
    # GlobCover is used to set difference between flooded pixel and pixel of permanent water body 
    expressions = ['' for i in range(S1.getNumBands())]
    thresholds = np.zeros(S1.getNumBands())
    S1_db = snappy.GPF.createProduct('LinearToFromdB', snappy.HashMap(), S1)
                   
    # loop through bands
    for bandNmbr in range(S1.getNumBands()):
        thresholds[i] = getThreshold(S1_db.getBandAt(bandNmbr))
        expressions[i] = 'if (%s < %s && land_cover_GlobCover != 210) then 1 else NaN' % (S1.getBandNames()[bandNmbrr], thresholds[bandNmbr])
        
    
    # do binarization
    S1_floodMask = binarization(GlobCover, expressions)
    print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)
    return S1_floodMask
    
    

In [18]:
import numpy as np 


ProcessedData = preprocess_sentinel_1(file_sentinel_product= file_sentinel)

FloodMask  = binary_flood_mask(S1 = ProcessedData)

1. Apply Orbit File:          --- 0.11  seconds ---
2. Thermal Noise Removal:     --- 0.01  seconds ---
3. Radiometric Calibration:    --- 0.04  seconds ---
4. Speckle Filtering:    --- 0.02  seconds ---
5. Terrain Correction:        --- 0.07  seconds ---
6. Binarization:              

MemoryError: 

In [23]:
import numpy as np
import gc

a = np.array([1,2,3])
del a
gc.collect()

0

# export resulting binarized image 

# Image processing 

In [ ]:
file_path = 'S1B_IW_GRDH_1SDV_20210625T094803_20210625T094828_027511_0348BD_AFA7.zip'
S1_source = snappy.ProductIO.readProduct(file_path)


In [ ]:
# read geographic coordinates from Sentinel-1 image meta data
meta_data = S1_source.getMetadataRoot().getElement('Abstracted_Metadata')

In [ ]:
meta_data

In [ ]:
# start processing button 
def on_processButton_clicked(b, S1_source, polygon_flex):
    # get coordinates and store in WKT format variable
    lng1, lat1 = polygon_flex.locations[0][0]['lng'], polygon_flex.locations[0][0]['lat']
    lng2, lat2 = polygon_flex.locations[0][1]['lng'], polygon_flex.locations[0][1]['lat']
    lng3, lat3 = polygon_flex.locations[0][2]['lng'], polygon_flex.locations[0][2]['lat']
    lng4, lat4 = polygon_flex.locations[0][3]['lng'], polygon_flex.locations[0][3]['lat']
    footprint = 'POLYGON((%s %s, %s %s, %s %s, %s %s, %s %s))' % (lng1, lat1, lng2, lat2, lng3, lat3, lng4, lat4, lng1, lat1)
    # run processing
    processing(S1_source, footprint)

In [ ]:
center = (meta_data.getAttributeDouble('centre_lat'), meta_data.getAttributeDouble('centre_lon'))
locations = [[{'lat' : meta_data.getAttributeDouble('first_near_lat'), 'lng' : meta_data.getAttributeDouble('first_near_long')},
              {'lat' : meta_data.getAttributeDouble('last_near_lat'),  'lng' : meta_data.getAttributeDouble('last_near_long')},
              {'lat' : meta_data.getAttributeDouble('last_far_lat'),   'lng' : meta_data.getAttributeDouble('last_far_long')},
              {'lat' : meta_data.getAttributeDouble('first_far_lat'),  'lng' : meta_data.getAttributeDouble('first_far_long')}]]

# creates interactive map
basic_map = ipyleaflet.Map(center = center, zoom = 7.5)
# defines fixed polygon illustrating Sentinel-1 image
polygon_fix = ipyleaflet.Polygon(locations = locations, color='royalblue')
basic_map.add_layer(polygon_fix)
# displays map
basic_map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
display(basic_map)

# editable polygon determining AOI
polygon_flex = ipyleaflet.Polygon(locations = locations,
                                  color = 'green',
                                  fill_color = 'green',
                                  transform = True)
basic_map.add_layer(polygon_flex)
# create process button and call function to start ptocessing when clicked
processButton = ipywidgets.Button(description = 'Start Processing')
processButton.on_click(functools.partial(on_processButton_clicked,
                                         S1_source = S1_source,
                                         polygon_flex = polygon_flex))
display(processButton)

In [ ]:
lng1, lat1 = polygon_flex.locations[0][0]['lng'], polygon_flex.locations[0][0]['lat']
lng2, lat2 = polygon_flex.locations[0][1]['lng'], polygon_flex.locations[0][1]['lat']
lng3, lat3 = polygon_flex.locations[0][2]['lng'], polygon_flex.locations[0][2]['lat']
lng4, lat4 = polygon_flex.locations[0][3]['lng'], polygon_flex.locations[0][3]['lat']
footprint = 'POLYGON((%s %s, %s %s, %s %s, %s %s, %s %s))' % (lng1, lat1, lng2, lat2, lng3, lat3, lng4, lat4, lng1, lat1)
footprint 

In [ ]:
import functools 

In [ ]:
locations 

In [ ]:
if polarisations == 'both':
    sourceBands = 'Amplitude_VH,Intensity_VH,Amplitude_VV,Intensity_VV'
    output_extensions   = 'processed_VHVV'
elif polarisations == 'VH':
    sourceBands = 'Amplitude_VH,Intensity_VH'
    output_extensions   = 'processed_VH'
elif polarisations == 'VV':
    sourceBands = 'Amplitude_VV,Intensity_VV'
    output_extensions   = 'processed_VV'




parameters = snappy.HashMap()
parameters.put('copyMetadata', True)
geom = snappy.WKTReader().read(footprint)
parameters.put('geoRegion', geom)
parameters.put('sourceBands', sourceBands)
S1_crop = snappy.GPF.createProduct('Subset', parameters, S1_source)

In [ ]:
S1_crop 

In [ ]:
geom 

In [ ]:
snappy.GPF.createProduct()

In [ ]:
def get_operating_system_name(operating_system):
    if any(keyWord in operating_system for keyWord in ['mac', 'Mac']):
        os_name = 'macos'
    elif any(keyWord in operating_system for keyWord in ['windows', 'Windows']):
        os_name = 'windows-x64'
    elif any(keyWord in operating_system for keyWord in ['Unix', 'unix','Linux','linux']):
        os_name = 'unix'
    return os_name

def get_installer_extension(operating_system):
    if operating_system == 'macos':
        installer_extension = '.dmg'
    elif operating_system.startswith('windows'):
        installer_extension = '.exe'
    elif operating_system == 'unix':
        installer_extension = '.sh'
    return installer_extension 

def get_snap_toolboxes(toolboxes):
    if 'sentinel' in toolboxes.lower():
        snap_toolboxes = 'sentinel'
    elif 'all' in toolboxes.lower():
        snap_toolboxes = 'all'
    return snap_toolboxes 



def get_download_link(operating_system = "macOS", toolbox = 'Sentinel Toolboxes'):
    os_name = get_operating_system_name(operating_system)
    installer_extension = get_installer_extension(os_name)
    snap_toolboxes = get_snap_toolboxes(toolbox)
    
    prefix_download_link = 'https://download.esa.int/step/snap/8.0/installers/esa-snap'
    download_link = '_'.join([prefix_download_link, snap_toolboxes,os_name, '8_0']) + installer_extension 
    return download_link


def download_ESA_SNAP(operating_system = "macOS", toolbox = 'Sentinel Toolboxes'):
    download_link = get_download_link(operating_system, toolbox)
    print("Trying to access: ", download_link)
    responce = requests.get(url = download_link)
    return responce 


In [ ]:
t1 = time()
responce = download_ESA_SNAP("mac", 'sentinel')
t2 = time()
print("Responce obtained after: ", t2-t1, 'seconds')

In [ ]:
from time import time 
time() - time()

In [ ]:
path_download_file = '/Users/mishaklein/Desktop/esa-snap_sentinel_macos_8_0.dmg'
t1 = time()
# --- write contents (the file) to disc ---
with open(path_download_file, "wb") as downloadFile:
    downloadFile.write(responce.content)
t2 = time()
print("Wrote file into: ", path_download_file)
print("saving file took: ", t2-t1, "seconds")